# Lesson 5: Human in the Loop

Note: This notebook is running in a later version of langgraph that it was filmed with. The later version has a couple of key additions:
- Additional state information is stored to memory and displayed when using `get_state()` or `get_state_history()`.
- State is additionally stored every state transition while previously it was stored at an interrupt or at the end.
These change the command output slightly, but are a useful addtion to the information available.

In [7]:
from dotenv import load_dotenv

_ = load_dotenv()

In [8]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [9]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [10]:
tool = TavilySearchResults(max_results=2)

## Manual human approval

In [11]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools) #Interreupts all available tools. Have to modify if any specific tools to be interrupted.

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [12]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [13]:
messages = [HumanMessage(content="Whats the weather in Liverpool?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in Liverpool?', id='c4801aa6-c1f9-44ec-946c-b377ce67c94d'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_DgERLtQNXfjLh3akxEAM64Sw', 'function': {'arguments': '{"query":"weather in Liverpool"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 152, 'total_tokens': 172}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-245ef508-6370-432e-b658-0f359320b321-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Liverpool'}, 'id': 'call_DgERLtQNXfjLh3akxEAM64Sw'}])]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_DgERLtQNXfjLh3akxEAM64Sw', 'function': {'arguments': '{"query":"weather in Liverpool"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage':

In [14]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Liverpool?', id='c4801aa6-c1f9-44ec-946c-b377ce67c94d'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Liverpool"}', 'name': 'tavily_search_results_json'}, 'id': 'call_DgERLtQNXfjLh3akxEAM64Sw', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 20, 'prompt_tokens': 152, 'total_tokens': 172}}, id='run-245ef508-6370-432e-b658-0f359320b321-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Liverpool'}, 'id': 'call_DgERLtQNXfjLh3akxEAM64Sw'}])]}, next=('action',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef2d6ed-55d7-647b-8001-a5e49b2808ba'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'llm': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'

In [15]:
abot.graph.get_state(thread).next

('action',)

### continue after interrupt

In [16]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Liverpool'}, 'id': 'call_DgERLtQNXfjLh3akxEAM64Sw'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Liverpool\', \'region\': \'Merseyside\', \'country\': \'United Kingdom\', \'lat\': 53.42, \'lon\': -2.94, \'tz_id\': \'Europe/London\', \'localtime_epoch\': 1718713959, \'localtime\': \'2024-06-18 13:32\'}, \'current\': {\'last_updated_epoch\': 1718713800, \'last_updated\': \'2024-06-18 13:30\', \'temp_c\': 14.8, \'temp_f\': 58.7, \'is_day\': 1, \'condition\': {\'text\': \'Cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/119.png\', \'code\': 1006}, \'wind_mph\': 10.1, \'wind_kph\': 16.2, \'wind_degree\': 302, \'wind_dir\': \'WNW\', \'pressure_mb\': 1017.0, \'pressure_in\': 30.02, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 69, \'cloud\': 64, \'feelslike_c\': 13.8, \'feelslike_f\': 56.8, \'windchill_

In [17]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Liverpool?', id='c4801aa6-c1f9-44ec-946c-b377ce67c94d'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Liverpool"}', 'name': 'tavily_search_results_json'}, 'id': 'call_DgERLtQNXfjLh3akxEAM64Sw', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 20, 'prompt_tokens': 152, 'total_tokens': 172}}, id='run-245ef508-6370-432e-b658-0f359320b321-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Liverpool'}, 'id': 'call_DgERLtQNXfjLh3akxEAM64Sw'}]), ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Liverpool\', \'region\': \'Merseyside\', \'country\': \'United Kingdom\', \'lat\': 53.42, \'lon\': -2.94, \'tz_id\': \'Europe/London\', \'lo

In [21]:
abot.graph.get_state(thread).next

('llm',)

## Modify State
Run until the interrupt and then modify the state.

In [23]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread),"\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='7ecb19b1-1a00-4442-9204-0e83e9647f7a'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iG9jBI364rOsazH02AKn0GDD', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-54e79a7d-e688-41d7-bc5c-3666b1764004-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_iG9jBI364rOsazH02AKn0GDD'}])]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iG9jBI364rOsazH02AKn0GDD', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': 

proceed?y
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles weather'}, 'id': 'call_MCFYNCBcXzzUi0xEhpuvE9ax'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.05, \'lon\': -118.24, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1718714099, \'localtime\': \'2024-06-18 5:34\'}, \'current\': {\'last_updated_epoch\': 1718713800, \'last_updated\': \'2024-06-18 05:30\', \'temp_c\': 17.8, \'temp_f\': 64.0, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 3.6, \'wind_kph\': 5.8, \'wind_degree\': 179, \'wind_dir\': \'S\', \'pressure_mb\': 1007.0, \'pressure_in\': 29.72, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 66, \'cloud\': 0, \'feelslike_c\': 17.8, \'feelslike

In [24]:
messages = [HumanMessage("Whats the weather in Cumilla?")]
thread = {"configurable": {"thread_id": "4"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in Cumilla?', id='70b4e069-47bc-4f50-901c-2130076722db'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI', 'function': {'arguments': '{"query":"weather in Cumilla"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a949c665-bf1f-4038-ac94-ff24eb405d6f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Cumilla'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI'}])]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI', 'function': {'arguments': '{"query":"weather in Cumilla"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'compl

In [25]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Cumilla?', id='70b4e069-47bc-4f50-901c-2130076722db'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Cumilla"}', 'name': 'tavily_search_results_json'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174}}, id='run-a949c665-bf1f-4038-ac94-ff24eb405d6f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Cumilla'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI'}])]}, next=('action',), config={'configurable': {'thread_id': '4', 'thread_ts': '1ef2d701-0d39-6ae3-8001-9e03ed1258fb'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'llm': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'functi

In [26]:
current_values = abot.graph.get_state(thread)

In [27]:
current_values.values['messages'][-1]

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Cumilla"}', 'name': 'tavily_search_results_json'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174}}, id='run-a949c665-bf1f-4038-ac94-ff24eb405d6f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Cumilla'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI'}])

In [28]:
current_values.values['messages'][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'weather in Cumilla'},
  'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI'}]

In [29]:
_id = current_values.values['messages'][-1].tool_calls[0]['id']
current_values.values['messages'][-1].tool_calls = [
    {'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Louisiana'},
  'id': _id}
]

In [30]:
abot.graph.update_state(thread, current_values.values)

{'messages': [HumanMessage(content='Whats the weather in Cumilla?', id='70b4e069-47bc-4f50-901c-2130076722db'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Cumilla"}', 'name': 'tavily_search_results_json'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174}}, id='run-a949c665-bf1f-4038-ac94-ff24eb405d6f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI'}])]}


{'configurable': {'thread_id': '4',
  'thread_ts': '1ef2d704-a7f1-67ea-8002-d4a68a68787c'}}

In [31]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Cumilla?', id='70b4e069-47bc-4f50-901c-2130076722db'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Cumilla"}', 'name': 'tavily_search_results_json'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174}}, id='run-a949c665-bf1f-4038-ac94-ff24eb405d6f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI'}])]}, next=('action',), config={'configurable': {'thread_id': '4', 'thread_ts': '1ef2d704-a7f1-67ea-8002-d4a68a68787c'}}, metadata={'source': 'update', 'step': 2, 'writes': {'llm': {'messages': [HumanMessage(content='Whats the weather in Cumilla?'

In [32]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Louisiana\', \'region\': \'Missouri\', \'country\': \'USA United States of America\', \'lat\': 39.44, \'lon\': -91.06, \'tz_id\': \'America/Chicago\', \'localtime_epoch\': 1718714562, \'localtime\': \'2024-06-18 7:42\'}, \'current\': {\'last_updated_epoch\': 1718713800, \'last_updated\': \'2024-06-18 07:30\', \'temp_c\': 24.5, \'temp_f\': 76.1, \'is_day\': 1, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/116.png\', \'code\': 1003}, \'wind_mph\': 13.6, \'wind_kph\': 22.0, \'wind_degree\': 170, \'wind_dir\': \'S\', \'pressure_mb\': 1015.0, \'pressure_in\': 29.96, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 82, \'cloud\': 50, \'feelslike_c\': 27.0, \'feelsl

## Time Travel

In [33]:
states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print('--')
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Cumilla?', id='70b4e069-47bc-4f50-901c-2130076722db'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Cumilla"}', 'name': 'tavily_search_results_json'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174}}, id='run-a949c665-bf1f-4038-ac94-ff24eb405d6f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI'}]), ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Louisiana\', \'region\': \'Missouri\', \'country\': \'USA United States of America\', \'lat\': 39.44, \'lon\': -91.06, \'tz_id\': \'Amer

To fetch the same state as was filmed, the offset below is changed to `-3` from `-1`. This accounts for the initial state `__start__` and the first state that are now stored to state memory with the latest version of software.

In [45]:
to_replay = states[-3]

In [46]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Cumilla?', id='70b4e069-47bc-4f50-901c-2130076722db'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Cumilla"}', 'name': 'tavily_search_results_json'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174}}, id='run-a949c665-bf1f-4038-ac94-ff24eb405d6f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Cumilla'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI'}])]}, next=('action',), config={'configurable': {'thread_id': '4', 'thread_ts': '1ef2d701-0d39-6ae3-8001-9e03ed1258fb'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'llm': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'functi

In [47]:
for event in abot.graph.stream(None, to_replay.config):
    for k, v in event.items():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Cumilla'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'error\': {\'code\': 1006, \'message\': \'No matching location found.\'}}"}, {\'url\': \'https://sharpweather.com/weather_cumilla/\', \'content\': \'Accurate weather forecast in Cumilla. En Es . Widgets for website . ... Chattogram Division . Weather Cumilla. Today Tomorrow 3 Days 7 Days 10 Days Weekend . Now 5:21 PM, Thursday 13 June, 2024 +87°F . Broken cloud sky . Wind: South, 12 mph. Pressure: 750 mmHg. Humidity: 73 %. ... Tue 18.06 +91°F +83°F . Wed 19.06 +89°F +82°F . More ...\'}]', name='tavily_search_results_json', id='d501f2ee-2015-49f3-8c4e-cf94fe1d8822', tool_call_id='call_Mph7rLxs1foZZwTYLXS4CWqI')]}
{'messages': [HumanMessage(content='Whats the weather in Cumilla?', id='70b4e069-47bc-4f50-901c-2130076722db'), AIMessage(content='', addi

## Go back in time and edit

In [48]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Cumilla?', id='70b4e069-47bc-4f50-901c-2130076722db'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Cumilla"}', 'name': 'tavily_search_results_json'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174}}, id='run-a949c665-bf1f-4038-ac94-ff24eb405d6f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Cumilla'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI'}])]}, next=('action',), config={'configurable': {'thread_id': '4', 'thread_ts': '1ef2d701-0d39-6ae3-8001-9e03ed1258fb'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'llm': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'functi

In [50]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']
to_replay.values['messages'][-1].tool_calls = [{'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in LA, accuweather'},
  'id': _id}]

In [51]:
branch_state = abot.graph.update_state(to_replay.config, to_replay.values)

{'messages': [HumanMessage(content='Whats the weather in Cumilla?', id='70b4e069-47bc-4f50-901c-2130076722db'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Cumilla"}', 'name': 'tavily_search_results_json'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174}}, id='run-a949c665-bf1f-4038-ac94-ff24eb405d6f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI'}])]}


In [52]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.05, \'lon\': -118.24, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1718714706, \'localtime\': \'2024-06-18 5:45\'}, \'current\': {\'last_updated_epoch\': 1718714700, \'last_updated\': \'2024-06-18 05:45\', \'temp_c\': 17.8, \'temp_f\': 64.0, \'is_day\': 1, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/113.png\', \'code\': 1000}, \'wind_mph\': 3.6, \'wind_kph\': 5.8, \'wind_degree\': 179, \'wind_dir\': \'S\', \'pressure_mb\': 1007.0, \'pressure_in\': 29.72, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 66, \'cloud\': 0, \'feelslike_c\': 17.8, \'feelsl

## Add message to a state at a given time

In [53]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Cumilla?', id='70b4e069-47bc-4f50-901c-2130076722db'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Cumilla"}', 'name': 'tavily_search_results_json'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174}}, id='run-a949c665-bf1f-4038-ac94-ff24eb405d6f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI'}])]}, next=('action',), config={'configurable': {'thread_id': '4', 'thread_ts': '1ef2d701-0d39-6ae3-8001-9e03ed1258fb'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'llm': {'messages': [AIMessage(content='', additional_kwargs={'tool_c

In [54]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']

In [55]:
state_update = {"messages": [ToolMessage(
    tool_call_id=_id,
    name="tavily_search_results_json",
    content="54 degree celcius",
)]}

In [56]:
branch_and_add = abot.graph.update_state(
    to_replay.config, 
    state_update, 
    as_node="action")

In [57]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in Cumilla?', id='70b4e069-47bc-4f50-901c-2130076722db'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Cumilla"}', 'name': 'tavily_search_results_json'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174}}, id='run-a949c665-bf1f-4038-ac94-ff24eb405d6f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Cumilla'}, 'id': 'call_Mph7rLxs1foZZwTYLXS4CWqI'}]), ToolMessage(content='54 degree celcius', name='tavily_search_results_json', id='90c2190a-1b10-40e0-9c21-378f6b61f70e', tool_call_id='call_Mph7rLxs1foZZwTYLXS4CWqI'), AIMessage(content='The current weather in Cumilla is 54 degrees Celsius. That seems unusually high. Would yo

# Extra Practice

## Build a small graph
This is a small simple graph you can tinker with if you want more insight into controlling state memory.

In [58]:
from dotenv import load_dotenv

_ = load_dotenv()

In [59]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langgraph.checkpoint.sqlite import SqliteSaver

Define a simple 2 node graph with the following state:
-`lnode`: last node
-`scratch`: a scratchpad location
-`count` : a counter that is incremented each step

In [60]:
class AgentState(TypedDict):
    lnode: str
    scratch: str
    count: Annotated[int, operator.add]

In [61]:
def node1(state: AgentState):
    print(f"node1, count:{state['count']}")
    return {"lnode": "node_1",
            "count": 1,
           }
def node2(state: AgentState):
    print(f"node2, count:{state['count']}")
    return {"lnode": "node_2",
            "count": 1,
           }

The graph goes N1->N2->N1... but breaks after count reaches 3.

In [62]:
def should_continue(state):
    return state["count"] < 3

In [63]:
builder = StateGraph(AgentState)
builder.add_node("Node1", node1)
builder.add_node("Node2", node2)

builder.add_edge("Node1", "Node2")
builder.add_conditional_edges("Node2", 
                              should_continue, 
                              {True: "Node1", False: END})
builder.set_entry_point("Node1")

In [64]:
memory = SqliteSaver.from_conn_string(":memory:")
graph = builder.compile(checkpointer=memory)

### Run it!
Now, set the thread and run!

In [65]:
thread = {"configurable": {"thread_id": str(1)}}
graph.invoke({"count":0, "scratch":"hi"},thread)

node1, count:0
node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

### Look at current state

Get the current state. Note the `values` which are the AgentState. Note the `config` and the `thread_ts`. You will be using those to refer to snapshots below.

In [66]:
graph.get_state(thread)

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef2d716-8f02-62c9-8004-98243ea800b9'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'count': 1, 'lnode': 'node_2'}}}, created_at='2024-06-18T12:50:58.689693+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef2d716-8efe-6fb5-8003-57a9ffd5f25e'}})

View all the statesnapshots in memory. You can use the displayed `count` agentstate variable to help track what you see. Notice the most recent snapshots are returned by the iterator first. Also note that there is a handy `step` variable in the metadata that counts the number of steps in the graph execution. This is a bit detailed - but you can also notice that the *parent_config* is the *config* of the previous node. At initial startup, additional states are inserted into memory to create a parent. This is something to check when you branch or *time travel* below.

### Look at state history

In [67]:
for state in graph.get_state_history(thread):
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef2d716-8f02-62c9-8004-98243ea800b9'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'count': 1, 'lnode': 'node_2'}}}, created_at='2024-06-18T12:50:58.689693+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef2d716-8efe-6fb5-8003-57a9ffd5f25e'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef2d716-8efe-6fb5-8003-57a9ffd5f25e'}}, metadata={'source': 'loop', 'step': 3, 'writes': {'Node1': {'count': 1, 'lnode': 'node_1'}}}, created_at='2024-06-18T12:50:58.688387+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef2d716-8efc-63de-8002-db72589ec4cd'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 2}, next=('Node1',), config={'configurable': {'thread_id': '1', 'thread_t

Store just the `config` into an list. Note the sequence of counts on the right. `get_state_history` returns the most recent snapshots first.

In [68]:
states = []
for state in graph.get_state_history(thread):
    states.append(state.config)
    print(state.config, state.values['count'])

{'configurable': {'thread_id': '1', 'thread_ts': '1ef2d716-8f02-62c9-8004-98243ea800b9'}} 4
{'configurable': {'thread_id': '1', 'thread_ts': '1ef2d716-8efe-6fb5-8003-57a9ffd5f25e'}} 3
{'configurable': {'thread_id': '1', 'thread_ts': '1ef2d716-8efc-63de-8002-db72589ec4cd'}} 2
{'configurable': {'thread_id': '1', 'thread_ts': '1ef2d716-8ef7-69fe-8001-564ec5188d15'}} 1
{'configurable': {'thread_id': '1', 'thread_ts': '1ef2d716-8ef3-6a09-8000-257051a380bb'}} 0
{'configurable': {'thread_id': '1', 'thread_ts': '1ef2d716-8ef0-6d4a-bfff-0e5a6b759563'}} 0


Grab an early state.

In [69]:
states[-3]

{'configurable': {'thread_id': '1',
  'thread_ts': '1ef2d716-8ef7-69fe-8001-564ec5188d15'}}

This is the state after Node1 completed for the first time. Note `next` is `Node2`and `count` is 1.

In [70]:
graph.get_state(states[-3])

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 1}, next=('Node2',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef2d716-8ef7-69fe-8001-564ec5188d15'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'Node1': {'count': 1, 'lnode': 'node_1'}}}, created_at='2024-06-18T12:50:58.685376+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef2d716-8ef3-6a09-8000-257051a380bb'}})

### Go Back in Time
Use that state in `invoke` to go back in time. Notice it uses states[-3] as *current_state* and continues to node2,

In [ ]:
graph.invoke(None, states[-3])

Notice the new states are now in state history. Notice the counts on the far right.

In [ ]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state.config, state.values['count'])

You can see the details below. Lots of text, but try to find the node that start the new branch. Notice the parent *config* is not the previous entry in the stack, but is the entry from state[-3].

In [ ]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state,"\n")

### Modify State
Let's start by starting a fresh thread and running to clean out history.

In [ ]:
thread2 = {"configurable": {"thread_id": str(2)}}
graph.invoke({"count":0, "scratch":"hi"},thread2)

In [ ]:
from IPython.display import Image

Image(graph.get_graph().draw_png())

In [ ]:
states2 = []
for state in graph.get_state_history(thread2):
    states2.append(state.config)
    print(state.config, state.values['count'])   

Start by grabbing a state.

In [ ]:
save_state = graph.get_state(states2[-3])
save_state

Now modify the values. One subtle item to note: Recall when agent state was defined, `count` used `operator.add` to indicate that values are *added* to the current value. Here, `-3` will be added to the current count value rather than replace it.

In [ ]:
save_state.values["count"] = -3
save_state.values["scratch"] = "hello"
save_state

Now update the state. This creates a new entry at the *top*, or *latest* entry in memory. This will become the current state.

In [ ]:
graph.update_state(thread2,save_state.values)

Current state is at the top. You can match the `thread_ts`.
Notice the `parent_config`, `thread_ts` of the new node - it is the previous node.

In [ ]:
for i, state in enumerate(graph.get_state_history(thread2)):
    if i >= 3:  #print latest 3
        break
    print(state, '\n')

### Try again with `as_node`
When writing using `update_state()`, you want to define to the graph logic which node should be assumed as the writer. What this does is allow th graph logic to find the node on the graph. After writing the values, the `next()` value is computed by travesing the graph using the new state. In this case, the state we have was written by `Node1`. The graph can then compute the next state as being `Node2`. Note that in some graphs, this may involve going through conditional edges!  Let's try this out.

In [ ]:
graph.update_state(thread2,save_state.values, as_node="Node1")

In [ ]:
for i, state in enumerate(graph.get_state_history(thread2)):
    if i >= 3:  #print latest 3
        break
    print(state, '\n')

`invoke` will run from the current state if not given a particular `thread_ts`. This is now the entry that was just added.

In [ ]:
graph.invoke(None,thread2)

Print out the state history, notice the `scratch` value change on the latest entries.

In [ ]:
for state in graph.get_state_history(thread2):
    print(state,"\n")

Continue to experiment!